# gameplay

> Fill in a module description here

In [ ]:
#| default_exp gameplay

In [ ]:
#| hide
from nbdev.showdoc import *

ModuleNotFoundError: No module named 'nbdev'

In [ ]:
%load_ext autoreload


In [ ]:
%autoreload 2

In [ ]:
#| export
import clue.model as model
import clue.ui as ui
from random import choice,shuffle,choices
import os

In [ ]:
#| hide
stgame = model.createClue(4)
stmoves = []

In [ ]:
#| export
def getPlay(pc,game, moves):
    
    

    ui.showBoard(pc,game,moves)
    playerRoom = model.roomOf(pc,game)
    passages = model.roomsAround(playerRoom)
    locs = [[3,x] for x in passages ]
    print("Move to Room:\n")
 
    l = ui.chooseKey(locs,pc,game,moves)
    l = passages[l-1]

    wKeys = [[1,x] for x in range(len(model.people))]
    p = ui.chooseKey(wKeys,1,game,moves) - 1

    wKeys = [[2,x] for x in range(len(model.weapons))]
    w = ui.chooseKey(wKeys,1,game,moves) - 1

    return p, w, l



In [ ]:
#| hide
stguess = getPlay(1,stgame,stmoves)
print(ui.showChoice(stguess))
def seenCounts(pc,game, moves):
    knoweledge = getSeen(pc,game, moves)
    p = [0,0,0]
    for x in knoweledge:
        i = int(x/100) -1
        p[i] += 1

    return p

In [ ]:
#| export




def simpleGuess(pc,game, moves):


    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    pdeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    p = choice(pdeck)%100
    
    pdeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    w = choice(pdeck)%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)


In [ ]:
#| export
def itemGuess(type,moves):
    if type == 1:
        l = len(model.people)
    if type == 2:
        l = len(model.weapons)
    if type == 3:
        l = len(model.rooms)
    ret = [1 for x in range(l)]
    for x in moves:
        ret[x[type]] += 1
    print(ret)
    return ret
    


def betterGuess(pc,game, moves):


    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    pdeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    pC = itemGuess(1,moves)
    pWeights = [pC[x%100] for x in  pdeck if x not in exclude]
    p = choices(pdeck,weights=pWeights)
    p = p[0]%100
    
    pdeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    pC = itemGuess(2,moves)
    pWeights = [pC[x%100] for x in  pdeck if x not in exclude]
    w = choices(pdeck,weights=pWeights)[0]%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)

## Visual

In [ ]:
#| export
def playerName(i):
    return ui.cardFull(model.cardLookup(1,i-1))
    
def runVisualGame(numPlayers, strats):
    done = False
    agame = model.createClue(4)
    amoves = []
    player = 1
    lastM = ["" for x in range(numPlayers)]
   
             
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        if isUser:
            #ui.printPlayerHand(player,agame)
            print(lastM[player-1])
        
        ans = strat(player,agame,amoves)
        if isUser:
            pass
            #os.system("clear")
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        print( playerName(player) + " suspects " + ui.showChoice(ans)+".")
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    print("\n\n gave a false one " +  ui.cardFull(model.keyLookUp(x))+"!")
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)
            if isUser:
                msg = playerName(check[-2]) + " has " + ui.cardFull(model.keyLookUp(card))+"!"
            else:
                msg = playerName(check[-2]) + " responds."
            print(msg)
            lastM[player-1] = msg
            
            player = player + 1
            if player - 1 >=len(strats):
               player = 1 
    
    print("\n" + playerName(player) + " won!")
    ui.printGame(agame) 

In [ ]:
#| hide
strategies = [("user",True,getPlay),("simple",False,simpleGuess),("better",False,betterGuess)]
runVisualGame(4,strategies)


╒══════════════╤═══════════════╤══════════════╕
│ 3) Ball Room │ Billiard Room │ Conservatory │
│    G         │               │    M         │
├──────────────┼───────────────┼──────────────┤
│ Study        │               │ Dining Room  │
│              │               │    W         │
├──────────────┼───────────────┼──────────────┤
│ 2) Library   │               │ Hall         │
│    S         │               │    P         │
├──────────────┼───────────────┼──────────────┤
│              │ 1) Lounge     │ Kitchen      │
│              │    P          │              │
╘══════════════╧═══════════════╧══════════════╛
Move to Room:

1 Lounge
2 Library
3 Ball Room


Choose from 1 to 3:  2


1 👠 Miss Scarlet
2 🧐 Colonel Mustard
3 ♡  Mrs. White
4 🐉 Mr. Green
5 🍆 Professor Plum
6 🦚 Mrs. Peacock


Choose from 1 to 6:  3


1*🔪 Knife
2 🔫 Revolver
3*🪢 Rope
4 🔧 Wrench
5*🕯️ Candlestick
6 🪠 Lead Pipe


Choose from 1 to 6:  2


👠 Miss Scarlet suspects ♡  Mrs. White with the 🔫 Revolver in the Library.
🧐 Colonel Mustard has Library!
🧐 Colonel Mustard suspects 🧐 Colonel Mustard with the 🔫 Revolver in the Billiard Room.
♡  Mrs. White responds.
[1, 3, 1, 1, 1, 1]


IndexError: list index out of range

In [ ]:
#| hide
printGame(agame) 

Solution
🦚 Mrs. Peacock, 🪢 Rope, Lounge

          [1] 👠 Miss Scarlet is in Library          
🔫 Revolver, 🔪 Knife, Hall, 🐉 Mr. Green, Ball Room

        [2] 🧐 Colonel Mustard is in Library         
Kitchen, Study, Conservatory, 🧐 Colonel Mustard, Billiard Room

       [3] ♡  Mrs. White is in Billiard Room        
🔧 Wrench, Dining Room, ♡  Mrs. White, Library

            [4] 🐉 Mr. Green is in Lounge            
🪠 Lead Pipe, 👠 Miss Scarlet, 🍆 Professor Plum, 🕯️ Candlestick


In [ ]:
#| hide
chooseCard(rooms,cardFull)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()